In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pylab as pl
import matplotlib.animation as animation
# This is how we can import src module from sibling module
import os, sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path: sys.path.append(nb_dir)
#######################
from src import wdata
from netCDF4 import Dataset
import seaborn as sns;
%matplotlib nbagg

def animate(images):
    """ need to keep a reference to returned object for the animation to work!"""
    fig = plt.figure()
    ims = []
    for n,i in enumerate(images):
        ims.append([plt.imshow(i, Animated=True, cmap='gray', origin='lower')])
    return animation.ArtistAnimation(fig, ims, interval=100, blit=True,
                                    repeat_delay=1000)

# A time-lapse through all 2017 radar images:

In [ ]:
# takes some time
# Todo: make it faster
%matplotlib nbagg
imgs2017 = wdata.Radar.getYear(2017)
ani = animate([i[1] for i in imgs2017])

# Average Rainfall

In [ ]:
y2017 = reduce(lambda x,y: x + y, [i[1] for i in imgs2017], np.zeros((126,201)))
plt.figure()
# plt.imshow(y2017/len(y2017), origin='lower', cmap='gray')
ax = sns.heatmap(y2017/len(y2017), cmap='gray', vmin=0, vmax=1)
ax.invert_yaxis()
# plt.show()

# Day-to-day Rainfall

In [ ]:
MONTHS = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
def rainfallDaily(year, month):
    days = map(lambda (hr, x): (hr[:-3], x), wdata.Radar.getMonth(year, month))
    days_rainfall = {k:0.0 for k in set([i[0] for i in days])}
    for (k,v) in days:
        days_rainfall[k] += np.sum(v) 
    xs = sorted(map(lambda x: int(x[-2:]), days_rainfall.keys()))
    ys = [days_rainfall['{:0>2d}-{:0>2d}-{:0>2d}'.format(year, month, i)] for i in xs]
    m, b = np.polyfit(xs, ys, 1) 
    plt.figure()
    plt.title('Daily Rainfall for {:s}, {:0>2d}'.format(MONTHS[month-1], year))
    plt.xlabel('Day')
    plt.ylabel('Rainfall(mm)')
    plt.plot(xs, ys)
    plt.scatter(xs, ys)
    plt.plot([-10, 100],[-10*m+b, 100*m+b])
    plt.xlim(0,32)
    plt.show()

rainfallDaily(2017, 6)
rainfallDaily(2017, 12)